This notebook is coded by Raunaq Chamdia for his capstone project for the IBM Data Science Course

# IBM Applied Data Science Capstone Course by Cousera (Week 5 Final Report)

### Opening a Chinese Restraunt in Sacramento, California

* Implementing web scraping to build Data Frame of neighboorhoods in Sacramento California 
* Using Nominatim to determining geographical coordinates of neighboorhoods
* Obtaining top 100 venues in each neighboorhood to explore and cluster the resulting data set
* Determining best cluster to open a chinese restraunt

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes 
!conda install -c conda-forge requests --yes 

from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Part 1

***Scraping HTML and Precessing HTML Raw Data***

In [37]:
scrapeData= requests.get('https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Sacramento,_California').text
soupData= BeautifulSoup(scrapeData,"html.parser").find_all('div', class_="mw-category-group")
neighRAW= []
for row in soupData:
    temp= row.find_all('li')
    for sub in temp:
        neighRAW.append(sub.text.replace(', Sacramento, California',''))

# Part 2

Determining Coordinates and creating dataframe based on Neighboorhood

In [69]:
coordsRAW = []
geolocator= Nominatim(user_agent="to_explorer")
for row in neighRAW:
    location= geolocator.geocode('{}, Sacramento, California'.format(row))
    if location: 
        coordsRAW.append([row, location.latitude, location.longitude])
coordsRAW

[['Ben Ali', 38.6155114, -121.4260401],
 ['Colonial Heights', 38.5324066, -121.4455093],
 ['Curtis Park', 38.5473589, -121.480279589834],
 ['Del Paso Heights', 38.6360152, -121.4202312],
 ['Downtown Sacramento', 38.57433815, -121.506844744067],
 ['East Sacramento', 38.5104289, -121.4862044],
 ['Elmhurst', 38.562591, -121.415031],
 ['Gardenland', 38.6110156, -121.4749548],
 ['Land Park', 38.5576836, -121.4885661],
 ['Meadowview', 38.4836062, -121.4671025],
 ['Midtown Sacramento', 38.7087912, -121.3866195],
 ['Natomas', 38.6527219, -121.497820581374],
 ['North Sacramento', 38.6060158, -121.4574543],
 ['Oak Park', 38.5512949, -121.4710657],
 ['Robla', 38.6612923, -121.4466209],
 ['Southside Park', 38.5685277, -121.502988582431],
 ['Tahoe Park', 38.5436373, -121.435289917803],
 ['Upper Land Park', 38.5576836, -121.4885661],
 ['Valley View Acres', 38.7124027, -121.0718889]]

In [127]:
sacramento_df= pd.DataFrame(coordsRAW, columns=["Neighborhood","Latitude", "Longitude"])
sacramento_df.head()

,Neighborhood,Latitude,Longitude
0,Ben Ali,38.615511,-121.426040
1,Colonial Heights,38.532407,-121.445509
2,Curtis Park,38.547359,-121.480280
3,Del Paso Heights,38.636015,-121.420231
4,Downtown Sacramento,38.574338,-121.506845


In [73]:
sacramento_df.shape

(19, 3)

# Part 3

#### Use geopy library to get the latitude and longitude values of Sacramento.

In [75]:
placeName= "Sacramento, California"
location= geolocator.geocode(placeName)
latitude= location.latitude
longitude= location.longitude
print('The geograpical coordinate of Scaramento are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scaramento are 38.5815719, -121.4943996.


#### Create a map of Sacramento with neighborhoods superimposed on top.

In [164]:
# create map of Sacramento using latitude and longitude values
map_sacramento = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(sacramento_df['Latitude'], sacramento_df['Longitude'],  sacramento_df['Neighborhood']):
    label = '{}, Sacramento, CA'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sacramento)  
    
map_sacramento

#### Define Foursquare Credentials and Version

In [82]:
# The code was removed by Watson Studio for sharing.

#### Exploring Neighborhoods in Sacramento

In [84]:
# Function from Lab
def getNearbyVenues(neighborhood, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for lat, lng,n in zip(latitudes, longitudes, neighborhood):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        for v in results:
            venues_list.append((
                n,
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']))

    nearby_venues = pd.DataFrame(venues_list)
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

In [89]:
LIMIT=100
sacramento_venues = getNearbyVenues(neighborhood= sacramento_df['Neighborhood'],
                                 latitudes=sacramento_df['Latitude'],
                                 longitudes=sacramento_df['Longitude']
                                  )

In [90]:
print("The API found {} venues in the Sacramento Neighborhoods".format(sacramento_venues.shape[0]))
sacramento_venues.head()

The API found 632 venues in the Sacramento Neighborhoods


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ben Ali,38.615511,-121.42604,River City Phoenix,38.610096,-121.432634,Marijuana Dispensary
1,Ben Ali,38.615511,-121.42604,Dolphin Scuba Center,38.610841,-121.431171,Sporting Goods Shop
2,Ben Ali,38.615511,-121.42604,Sarom's Southern Kitchen,38.611439,-121.422809,Southern / Soul Food Restaurant
3,Ben Ali,38.615511,-121.42604,RT Metro,38.616119,-121.431389,Train Station
4,Ben Ali,38.615511,-121.42604,7-Eleven,38.620045,-121.419002,Convenience Store


In [91]:
print('There are {} uniques categories.'.format(len(sacramento_venues['Venue Category'].unique())))

There are 164 uniques categories.


#### Analyzing Each Neighborhood

In [93]:
# one hot encoding
sacramento_onehot = pd.get_dummies(sacramento_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sacramento_onehot['Neighborhoods'] = sacramento_venues['Neighborhood'] 

# move Neighborhoods to the first column
fixed_columns = list(sacramento_onehot.columns[-1:]) + list(sacramento_onehot.columns[:-1])
sacramento_onehot = sacramento_onehot[fixed_columns]

sacramento_onehot.head()

,Neighborhoods,ATM,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Big Box Store,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Football Field,College Gym,College Rec Center,Comedy Club,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,General College & University,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,History Museum,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Laundromat,Lawyer,Light Rail Station,Liquor Store,Marijuana Dispensary,Market,Martial Arts Dojo,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Other Repair Shop,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Record Shop,Rental Car Location,Rental Service,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Street Art,Street Food Gathering,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Train,Train Station,Video Store,Vietnamese Restaurant,Wings Joint
0,Ben Ali,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ben Ali,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ben Ali,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ben Ali,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Ben Ali,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [118]:
sacramento_grouped = sacramento_onehot.groupby(['Neighborhoods']).mean().reset_index()
sacramento_grouped

,Neighborhoods,ATM,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Big Box Store,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Football Field,College Gym,College Rec Center,Comedy Club,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,General College & University,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,History Museum,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Laundromat,Lawyer,Light Rail Station,Liquor Store,Marijuana Dispensary,Market,Martial Arts Dojo,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Other Repair Shop,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Record Shop,Rental Car Location,Rental Service,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Street Art,Street Food Gathering,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Train,Train Station,Video Store,Vietnamese Restaurant,Wings Joint
0,Ben Ali,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.00,0.058824,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.117647,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.058824,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.058824,0.117647,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.058824,0.176471,0.000000,0.000000,0.00000,0.058824,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.058824,0.000000,0.000000,0.000000
1,Colonial Heights,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.029412,0.000000,0.000000,0.00,0.000000,0.000000,0.

#### Clustering Neighborhoods

Run *k*-means to cluster the neighborhood into 3 clusters.

In [131]:
# set number of clusters
kclusters = 3
sacramento_grouped_clustering = sacramento_grouped[['Chinese Restaurant']]

# run k-means clustering
kmeans = KMeans(init="k-means++",n_clusters=kclusters, random_state=0).fit(sacramento_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 1, 2, 2, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0, 1, 2, 1, 0],
      dtype=int32)

Creating a new dataframe that includes the cluster label, restraunt mean and latitude and longitude for each neighborhood

In [152]:
sacramento_merged = sacramento_df.copy().drop(['Longitude','Latitude'],1)
sacramento_merged["No. of Chinese Restraunts"]=sacramento_grouped[['Chinese Restaurant']]
sacramento_merged["Cluster Labels"]=kmeans.labels_
sacramento_merged["Latitude"]=sacramento_df[["Latitude"]]
sacramento_merged["Longitude"]=sacramento_df[["Longitude"]]
sacramento_merged= sacramento_merged.sort_values(["Cluster Labels"]).reset_index().drop('index',1)
sacramento_merged

,Neighborhood,No. of Chinese Restraunts,Cluster Labels,Latitude,Longitude
0,Meadowview,0.000000,0,38.483606,-121.467102
1,Robla,0.000000,0,38.661292,-121.446621
2,North Sacramento,0.000000,0,38.606016,-121.457454
3,Natomas,0.000000,0,38.652722,-121.497821
4,East Sacramento,0.000000,0,38.510429,-121.486204
5,Valley View Acres,0.000000,0,38.712403,-121.071889
6,Gardenland,0.038462,1,38.611016,-121.474955
7,Land Park,0.039474,1,38.557684,-121.488566
8,Upper Land Park,0.039474,1,38.557684,-121.488566
9,Midtown Sacramento,0.040000,1,38.708791,-121.386619


#### Cluster Visual

In [163]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sacramento_merged['Latitude'], sacramento_merged['Longitude'], sacramento_merged['Neighborhood'], sacramento_merged['Cluster Labels']):
    label = folium.Popup('{}, Sacramento CA - Cluster {}'.format(poi,cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examining Clusters

Cluster 0 (No Chinese Restraunts)

In [157]:
sacramento_merged.loc[sacramento_merged['Cluster Labels'] == 0]

,Neighborhood,No. of Chinese Restraunts,Cluster Labels,Latitude,Longitude
0,Meadowview,0.0,0,38.483606,-121.467102
1,Robla,0.0,0,38.661292,-121.446621
2,North Sacramento,0.0,0,38.606016,-121.457454
3,Natomas,0.0,0,38.652722,-121.497821
4,East Sacramento,0.0,0,38.510429,-121.486204
5,Valley View Acres,0.0,0,38.712403,-121.071889


Cluster 1 (Low density of Chinese Restraunts)

In [158]:
sacramento_merged.loc[sacramento_merged['Cluster Labels'] == 1]

,Neighborhood,No. of Chinese Restraunts,Cluster Labels,Latitude,Longitude
6,Gardenland,0.038462,1,38.611016,-121.474955
7,Land Park,0.039474,1,38.557684,-121.488566
8,Upper Land Park,0.039474,1,38.557684,-121.488566
9,Midtown Sacramento,0.040000,1,38.708791,-121.386619
10,Oak Park,0.048780,1,38.551295,-121.471066
11,Colonial Heights,0.029412,1,38.532407,-121.445509
12,Southside Park,0.042254,1,38.568528,-121.502989
13,Downtown Sacramento,0.020000,1,38.574338,-121.506845


Cluster 2 (High Density of Chinese Restraunts)

In [159]:
sacramento_merged.loc[sacramento_merged['Cluster Labels'] == 2]

,Neighborhood,No. of Chinese Restraunts,Cluster Labels,Latitude,Longitude
14,Elmhurst,0.062500,2,38.562591,-121.415031
15,Del Paso Heights,0.100000,2,38.636015,-121.420231
16,Curtis Park,0.065217,2,38.547359,-121.480280
17,Tahoe Park,0.076923,2,38.543637,-121.435290
18,Ben Ali,0.058824,2,38.615511,-121.426040


## Observations:

It is evident that the majority of the Chinese restaurants are concentrated in the middle of the city of Sacramento. The highest density if Chinese restaurants is seen in Cluster 2, followed by moderate density of Chinese restaurants in cluster 1 and no density of Chinese restaurants in cluster 0.

Hence while choosing a neighborhood to open a Chinese restaurant it would be wise to consider locations in cluster 0 due to lack of competition and ability to conquer the market in that area.

